In [ ]:
import torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
import imageio


In [ ]:
batch_size = 128

data_loader = DataLoader(
  datasets.MNIST('./data/mnist', 
      train=True, 
      download=True, 
      transform=transforms.Compose([transforms.ToTensor()])),
  batch_size=batch_size,
  shuffle=True, num_workers=2)

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


La classe qui defini le Générateur (un reseau de neuronnes) :

In [ ]:

class generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, dataset='mnist'):
        super(generator, self).__init__()
        self.nc = 1
        self.nz = 100
        self.input_height = 28
        self.input_width = 28

        self.fc = nn.Sequential(
            nn.Linear(self.nz, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_height // 4) * (self.input_width // 4)),
            nn.BatchNorm1d(128 * (self.input_height // 4) * (self.input_width // 4)),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.nc, 4, 2, 1),
            nn.Sigmoid(),
        )
    def forward(self, input):
        x = self.fc(input)
        x = x.view(-1, 128, (self.input_height // 4), (self.input_width // 4))
        x = self.deconv(x)
        return x

La classe qui defini le Discriminateur (aussi un reseau de neuronnes) :

In [ ]:
class discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    def __init__(self, dataset = 'mnist'):
        super(discriminator, self).__init__()
        self.input_dim = 1
        self.input_height = 28
        self.input_width = 28
        self.output_dim = 1


        self.conv = nn.Sequential(
            nn.Conv2d(self.input_dim, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * (self.input_height // 4) * (self.input_width // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, self.output_dim),
            nn.Sigmoid(),
        )

    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, 128 * (self.input_height // 4) * (self.input_width // 4))
        x = self.fc(x)
        return x


La classe qui defini le GAN (qui contient donc le générateur et le discriminateur). Vous devrez completer la fonction train.

In [ ]:
class GAN(object):
    def __init__(self, batch_size):
        # parameters
        self.batch_size = batch_size
        self.lrD = 0.1
        self.lrG = 0.1
        self.noise = 0.001
        self.epoch = 50
        self.gpu_mode = torch.cuda.is_available()
        
        # networks init
        self.G = generator()
        self.D = discriminator()

        self.G_optimizer = optim.SGD(self.G.parameters(), lr=self.lrG)
        self.D_optimizer = optim.SGD(self.D.parameters(), lr=self.lrD)

        self.G.apply(self.weights_init)
        self.D.apply(self.weights_init)
# 
        if self.gpu_mode:
            self.G.cuda()
            self.D.cuda()
            self.BCE_loss = nn.BCELoss().cuda()
            self.y_real_ = torch.ones(self.batch_size,1).cuda()
            self.y_fake_ = torch.zeros(self.batch_size,1).cuda()
        else:
            self.BCE_loss = nn.BCELoss()
            self.y_real_ = torch.ones(self.batch_size,1)
            self.y_fake_ = torch.zeros(self.batch_size,1)

    def train(self,epoch):

        self.D.train()
        self.G.train()

        for epoch in range(self.epoch):

          print(epoch,"/", self.epoch)

          for i , (x_, _) in enumerate(data_loader):
              ##########################
              #  Discriminator update  #
              ##########################
              self.D_optimizer.zero_grad()
              out1=self.D(x_)
              #donnees generees en batch
              z=np.random.rand(self.batch_size,200)
              g_z=self.G(z)
              out2=self.D.forward(g_z)
              loss1o=self.BCE_loss(out1,[1]*len(out1))
              loss2o=self.BCE_loss(out2,[0]*len(out2))
              loss=loss1o+loss2o
              loss.backward()
              self.D_optimizer.step()
              
              ##########################
              #    Generator update    #
              ##########################
              
              noise2=np.random(batch_size,100)
              noise2=torch.from_numpy(noise2).cuda()
              n=self.G(noise2)
              g=self.D(n)
              lossG=self.BCE_loss(d,self.y_real_)
              lossG.backward()
              self.G_optimizer.step()
              self.G_optimizer.zero_grad()
              if i%100==0:
                  print("Iteration ", i,"/", len(self.data_loader), "D_loss : ",losses['D'], " - G_loss : ", losses['G'])


    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)
model=GAN(128)
model.train(1)

0 / 50


AttributeError: ignored

Recherchez comment visualiser (et enregistrer) les images que votre générateur entrainé produit.

In [ ]:
#Visualisation 

#A compléter